In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
sys.path.append('utils/')

import keras
from keras import optimizers

# use non standard flow_from_directory
from image_preprocessing_ver2 import ImageDataGenerator
# it outputs y_batch that contains onehot targets and logits
# logits came from xception

from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Lambda, concatenate, Activation
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras import backend as K

from mobilenet import get_mobilenet

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
train_logits = np.load(data_dir + 'train_logits.npy')[()]
val_logits = np.load(data_dir + 'val_logits.npy')[()]

In [5]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

# note: i'm also passing dicts of logits
train_generator = data_generator.flow_from_directory(
    data_dir + 'train', train_logits,
    target_size=(299, 299),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', val_logits,
    target_size=(299, 299),
    batch_size=64
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


# Create model

In [6]:
temperature = 7
lambda_const = 0.01

In [7]:
model = get_mobilenet()

for layer in model.layers[:-7]:
    layer.trainable = False

# remove softmax
model.layers.pop()

# usual probabilities
logits = model.layers[-1].output
probabilities = Activation('softmax')(logits)

# softed probabilities
logits_T = Lambda(lambda x: x/temperature)(logits)
probabilities_T = Activation('softmax')(logits_T)

output = concatenate([probabilities, probabilities_T])
model = Model(model.input, output)
# now model outputs 512 dimensional vectors

# Create custom loss

In [8]:
def knowledge_distillation_loss(y_true, y_pred):    
    
    # split in 
    #    onehot hard true targets
    #    logits from xception
    y_true, logits = y_true[:, :256], y_true[:, 256:]
    
    # convert logits to soft targets
    y_soft = K.softmax(logits/temperature)
    
    # split in 
    #    usual output probabilities
    #    probabilities made softer with temperature
    y_pred, y_pred_soft = y_pred[:, :256], y_pred[:, 256:]    
    
    return lambda_const*logloss(y_true, y_pred) + logloss(y_soft, y_pred_soft)

# For testing use usual output probabilities (without temperature)

In [9]:
def accuracy(y_true, y_pred):
    y_true = y_true[:, :256]
    y_pred = y_pred[:, :256]
    return categorical_accuracy(y_true, y_pred)

In [10]:
def top_5_accuracy(y_true, y_pred):
    y_true = y_true[:, :256]
    y_pred = y_pred[:, :256]
    return top_k_categorical_accuracy(y_true, y_pred)

# Train

In [11]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-4), 
    loss=knowledge_distillation_loss, metrics=[accuracy, top_5_accuracy]
)

In [12]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=400, epochs=15, verbose=1,
    validation_data=val_generator, validation_steps=48 
)

Epoch 1/15
400/400 [==============================] - 86s - loss: 5.8417 - accuracy: 0.0788 - top_5_accuracy: 0.1972 - val_loss: 5.7050 - val_accuracy: 0.3141 - val_top_5_accuracy: 0.5898
Epoch 2/15
400/400 [==============================] - 85s - loss: 5.6509 - accuracy: 0.2938 - top_5_accuracy: 0.5495 - val_loss: 5.6104 - val_accuracy: 0.4870 - val_top_5_accuracy: 0.7529
Epoch 3/15
400/400 [==============================] - 85s - loss: 5.5930 - accuracy: 0.4237 - top_5_accuracy: 0.7006 - val_loss: 5.5798 - val_accuracy: 0.5452 - val_top_5_accuracy: 0.8001
Epoch 4/15
400/400 [==============================] - 85s - loss: 5.5750 - accuracy: 0.4810 - top_5_accuracy: 0.7534 - val_loss: 5.5703 - val_accuracy: 0.5492 - val_top_5_accuracy: 0.8053
Epoch 5/15
400/400 [==============================] - 85s - loss: 5.5685 - accuracy: 0.5051 - top_5_accuracy: 0.7691 - val_loss: 5.5658 - val_accuracy: 0.5680 - val_top_5_accuracy: 0.8275
Epoch 6/15
400/400 [==============================] - 85s - 

# Results

In [13]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', val_logits,
    target_size=(299, 299),
    batch_size=64, shuffle=False
)

Found 5120 images belonging to 256 classes.


In [14]:
model.evaluate_generator(val_generator_no_shuffle, 80)

[5.5564268946647646, 0.60292968749999998, 0.84687500000000004]